In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recipe-for-rating-predict-food-ratings-using-ml/sample.csv
/kaggle/input/recipe-for-rating-predict-food-ratings-using-ml/train.csv
/kaggle/input/recipe-for-rating-predict-food-ratings-using-ml/test.csv


In [7]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
data_train=pd.read_csv('/kaggle/input/recipe-for-rating-predict-food-ratings-using-ml/train.csv')
data_test=pd.read_csv('/kaggle/input/recipe-for-rating-predict-food-ratings-using-ml/test.csv')
data_sample=pd.read_csv('/kaggle/input/recipe-for-rating-predict-food-ratings-using-ml/sample.csv')

In [8]:
data_train.shape

(13636, 15)

In [9]:
data_train.dtypes

ID                    int64
RecipeNumber          int64
RecipeCode            int64
RecipeName           object
CommentID            object
UserID               object
UserName             object
UserReputation        int64
CreationTimestamp     int64
ReplyCount            int64
ThumbsUpCount         int64
ThumbsDownCount       int64
Rating                int64
BestScore             int64
Recipe_Review        object
dtype: object

In [12]:
X_train = data_train.drop("Rating", axis=1)
y_train = data_train['Rating']

# Preprocessing pipeline
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_features = X_train.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Feature selection pipeline
feature_selection = SelectKBest(chi2)

# Random Forest classifier
classifier = RandomForestClassifier()

# Pipeline combining preprocessing, feature selection, and classifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', feature_selection),
    ('classifier', classifier)
])

# Parameter grid for GridSearchCV
param_grid = {
    'feature_selection__k': ['all'],
    'classifier__n_estimators': [100],
    'classifier__max_depth': [None],
    'classifier__min_samples_split': [2],
    'classifier__min_samples_leaf': [1],
    'classifier__bootstrap': [False],
    'classifier__criterion': ['gini'],
    'classifier__max_features': ['sqrt'],
    'classifier__class_weight': [None],
    'classifier__random_state': [42],
    'classifier__verbose': [0],
    'classifier__warm_start': [False]
}

# Grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=2, scoring='accuracy', verbose=3, error_score='raise')
grid_search.fit(X_train, y_train)

# Get the best model and best score
best_model = grid_search.best_estimator_
best_score = grid_search.best_score_

print("Best Model:", best_model)
print("Best Score:", best_score)

# Make predictions on test data
y_pred = best_model.predict(data_test)

# Create submission DataFrame
submission = pd.DataFrame({"ID": range(1, len(y_pred) + 1), "Rating": y_pred})

# Save submission to CSV file
submission.to_csv('submission.csv', index=False)



Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END classifier__bootstrap=False, classifier__class_weight=None, classifier__criterion=gini, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=100, classifier__random_state=42, classifier__verbose=0, classifier__warm_start=False, feature_selection__k=all;, score=0.769 total time=  16.7s
[CV 2/2] END classifier__bootstrap=False, classifier__class_weight=None, classifier__criterion=gini, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=100, classifier__random_state=42, classifier__verbose=0, classifier__warm_start=False, feature_selection__k=all;, score=0.769 total time=  17.0s
Best Model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                 